In [1]:
import plotly.express as px
import pandas as pd
import os
from datetime import datetime, timedelta

In [11]:
px.set_mapbox_access_token(os.environ['MAPBOX_TOKEN'])

In [2]:
df = pd.read_csv('data/worldcities.csv').sort_values('population', ascending=False)
df['size'] = 15
df.rename(columns={'lng': 'lon'}, inplace=True)
df.sample()

,city,city_ascii,lat,lon,country,iso2,iso3,admin_name,capital,population,id,size
8581,La Palma,La Palma,8.3982,-78.1402,Panama,PA,PAN,Darién,admin,1845.0,1591177082,15


In [3]:
us = df[df['country'] == 'United States'].iloc[:25]
indo = df[df['country'] == 'Indonesia'][:20]
china = df[df['country'] == 'China'].iloc[:15]
russia = df[df['country'] == 'Russia'].iloc[:10]
france = df[df['country'] == 'France'].iloc[:5]
bahamas = df[df['country'] == 'Bahamas, The'].iloc[:4]

In [9]:
now = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

In [ ]:
if size == 25:
    zoom = 3
if size == 20:
    zoom = 4
if size == 15:
    zoom = 5
if size == 10:
    zoom = 6


In [22]:
fig = px.scatter_mapbox(bahamas, hover_data=['population'],
                            lat='lat', lon='lon',
                            size='size',
                            color_continuous_scale=px.colors.cyclical.IceFire,
                            zoom=5)
fig.update_traces(textposition='top center')
fig.update_layout(title_text=f'Temperatures for {now}', title_x=0.5)
fig.show()

In [5]:
def map_plot(df):
    """
    A function to plot a scatter_mapbox
    of plotly
    :param country: a country input by user
    :param df: pandas.DataFrame containing temperature
               and cities data
    :return: plotly figure
    """
    # Get time for the moment
    # Construct the figure
    fig = px.scatter_mapbox(us, hover_data=['population'],
                            lat='lat', lon='lon',
                            size='size',
                            color_continuous_scale=px.colors.cyclical.IceFire,
                            zoom=5)
    fig.update_traces(textposition='top center')
    fig.update_layout(title_text=f'Temperatures for {now}', title_x=0.5)

    return fig